In [ ]:
!pip install --upgrade -q gspread

from google.colab.patches import cv2_imshow
from google.colab import auth
import gspread
import pandas as pd
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor
from skimage import io, transform
from tqdm import tqdm
auth.authenticate_user()
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 755.2 kB/s eta 0:00:00
ERROR: Operation cancelled by user


KeyboardInterrupt: ignored

In [ ]:
import shutil
import os
from google.colab import drive
drive.mount('/content/drive')

worksheet = gc.open('hands and palms loras').worksheet('scale1').get("A2:I625")
df = pd.DataFrame.from_records(worksheet)
df = df.rename({0:'rank', 1:'checkpoint', 2:'side',3:'gender', 4:'hand', 5:'image', 6: 'bad', 7:'ok', 8:'good'}, axis=1)
df = df.loc[df['bad']=="1"]

for index, row in tqdm([(index, row) for (index, row) in df.iterrows()]):
  if row['rank'] != 'None':
    path = f'/content/drive/MyDrive/LoRA/inference/hands and palms r{row["rank"]} lora/{row["side"]}_of_fair-skinned_{row["gender"]}_{row["hand"]}_hand_chkpt{row["checkpoint"]}_steps30_{row["image"]}.png'
  else:
    path = f'/content/drive/MyDrive/LoRA/inference/hands and palms r{row["rank"]} lora/{row["side"]}_of_fair-skinned_{row["gender"]}_{row["hand"]}_hand_chkpt5000_steps30_{row["image"]}.png'
  shutil.copy(path, '/content/drive/MyDrive/cs236 project/anybadhand/'+os.path.basename(path))
  shutil.copy(path, '/content/drive/MyDrive/cs236 project/bad11k/'+os.path.basename(path))

In [ ]:
!ls "/content/drive/MyDrive/cs236 project/anybadhand"

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pip install git+https://github.com/huggingface/diffusers
!git clone https://github.com/huggingface/diffusers.git
%cd diffusers/examples/textual_inversion

Mounted at /content/drive
  Cloning https://github.com/huggingface/diffusers to /tmp/pip-req-build-f1btfpkt
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers /tmp/pip-req-build-f1btfpkt
  Resolved https://github.com/huggingface/diffusers to commit ab6672fecd45aef292c69476aeb0275a2b5b418b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.25.0.dev0-py3-none-any.whl size=1775571 sha256=cfe3e78239df40f6d7f2307569984caf0c38f0378ae48fb071684584ef4c2629
  Stored in directory: /tmp/pip-ephem-wheel-cache-zb85dkmg/wheels/f7/7d/99/d361489e5762e3464b3811bc629e94cf5bf5ef44dd5c3c4d52
Successfully built diffusers
Cloning into 'diffusers'...
remote: Enumerating objects: 47451, done.
remote: Counting objects: 100% (1101/1101), done.
remote: Compressing objects: 100% (434/434), done.
remote: Total 47451 (delta 745), reused 

In [2]:
!pip install -r requirements.txt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 5.8 MB/s eta 0:00:00


In [ ]:

!accelerate launch textual_inversion.py \
  --pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5" \
  --train_data_dir='/content/drive/MyDrive/cs236 project cornell/anybadhand/' \
  --learnable_property="object" \
  --placeholder_token="<anybadhand>" \
  --initializer_token="hand" \
  --resolution=512 \
  --train_batch_size=4 \
  --gradient_accumulation_steps=1 \
  --max_train_steps=3000 \
  --learning_rate=5.0e-04 \
  --scale_lr \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --output_dir="/content/drive/MyDrive/cs236 project cornell/textual_inversion_any_bad_hand"

In [ ]:

!accelerate launch textual_inversion.py \
  --pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5" \
  --train_data_dir='/content/drive/MyDrive/cs236 project cornell/badcontexthand/' \
  --learnable_property="object" \
  --placeholder_token="<badcontexthand>" \
  --initializer_token="hand" \
  --resolution=512 \
  --train_batch_size=4 \
  --gradient_accumulation_steps=1 \
  --max_train_steps=3000 \
  --learning_rate=5.0e-04 \
  --scale_lr \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --output_dir="/content/drive/MyDrive/cs236 project cornell/textual_inversion_badcontexthand"

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2023-12-06 01:05:53.535080: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-06 01:05:53.535139: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-06 01:05:53.535179: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has alr

In [3]:

!accelerate launch textual_inversion.py \
  --pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5" \
  --train_data_dir='/content/drive/MyDrive/cs236 project cornell/bad11k/' \
  --learnable_property="object" \
  --placeholder_token="<bad11khand>" \
  --initializer_token="hand" \
  --resolution=512 \
  --train_batch_size=4 \
  --gradient_accumulation_steps=1 \
  --max_train_steps=3000 \
  --learning_rate=5.0e-04 \
  --scale_lr \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --output_dir="/content/drive/MyDrive/cs236 project cornell/textual_inversion_bad11khand" \
  --resume_from_checkpoint='latest'

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2023-12-06 11:35:35.883541: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-06 11:35:35.883606: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-06 11:35:35.883639: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has alr